# Поиск коллокаций для оценочных слов
С помощью nltk.collocations

In [2]:
import collections
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

from preprocessing import tokenize

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

### Загрузка корпуса рецензий

In [3]:
with open("SentiRuEval_rest_train.xml", encoding = "utf-8") as f:
    soup = BeautifulSoup(f, "lxml")

reviews = []
for review in soup.find_all("review"):
    text = review.find("text").text
    reviews.append(text)

In [5]:
print(len(reviews))

19034


### Предобработка текстов
Токенизация и стопслова. Можно добавить лемматизацию (но пока без неё из-за времени работы на большом объёме корпуса).

In [6]:
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    clean_text = re.sub('[0-9]+', ' ', clean_text)
    return clean_text

In [7]:
stop_words = stopwords.words("russian")

In [8]:
reviews_tokenized = []
for review in reviews:
    words = preprocessing(review).lower().split()
    lemmas = []
    for word in words:
#         p = morph.parse(word)[0]
#         lemmas.append(p.normal_form)
        if word not in stop_words and len(word) > 1:
            lemmas.append(word)
    reviews_tokenized.append(lemmas)

In [9]:
print(reviews_tokenized[0])

['пускай', 'обижается', 'наш', 'прославленный', 'защитник', 'франкофон', 'монреаль', 'канадиенс', 'maxime', 'некоторой', 'опаской', 'относился', 'этому', 'народу', 'народу', 'способному', 'таким', 'благоговением', 'доводить', 'цирроза', 'пернатую', 'живность', 'заставлять', 'специальных', 'поисковых', 'свиней', 'копошиться', 'грязи', 'поисках', 'сумчатых', 'грибов', 'ковыряться', 'тине', 'собирая', 'брюхоногих', 'двустворчатых', 'народу', 'подсадившего', 'население', 'земного', 'шара', 'муть', 'именуемое', 'божоле', 'нуво', 'сыры', 'плесенью', 'квакающих', 'жаб', 'скачущих', 'болотам', 'менее', 'единожды', 'заглянув', 'мог', 'успокоиться', 'пока', 'вкусил', 'большую', 'часть', 'меню', 'буду', 'вдаваться', 'детали', 'вкусно', 'невкусно', 'приведу', 'турнирную', 'таблицу', 'коли', 'начали', 'хоккея', 'петух', 'запеченный', 'сливках', 'картофелем', 'грибами', 'почему', 'петух', 'курица', 'различать', 'научился', 'домашний', 'пирог', 'яблочный', 'горячий', 'тающий', 'говядина', 'бургунски'

### Поиск коллокаций

In [29]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents(reviews_tokenized)

finder.nbest(bigram_measures.likelihood_ratio, 10)

[('день', 'рождения'),
 ('молодой', 'человек'),
 ('очень', 'вкусно'),
 ('могу', 'сказать'),
 ('очень', 'понравилось'),
 ('живая', 'музыка'),
 ('отдельное', 'спасибо'),
 ('сих', 'пор'),
 ('крайней', 'мере'),
 ('самом', 'деле')]

Можно взять ещё и триграммы, но занимает много времени.

In [30]:
# trigram_measures = nltk.collocations.TrigramAssocMeasures()
# finder_tri = nltk.collocations.TrigramCollocationFinder.from_documents(reviews_tokenized)
# finder_tri.nbest(trigram_measures.likelihood_ratio, 10)

### Поиск топ 5 биграмм для расширенного списка оценочных слов 

In [38]:
def find_bigramms_for_word(word):
    word_filter = lambda *w: word not in w
    
    finder = nltk.collocations.BigramCollocationFinder.from_documents(reviews_tokenized)
    finder.apply_ngram_filter(word_filter)
    
    return finder.nbest(bigram_measures.likelihood_ratio, 5)

In [39]:
with open('w2v_all_clean.txt', encoding = "utf-8") as file:
    target_words = [line.strip() for line in file.readlines()]

In [42]:
len(target_words)

128

In [ ]:
target_bigramms = collections.defaultdict()
for word in target_words:
    target_bigramms[word] = find_bigramms_for_word(word)

### Вывод в файл

In [45]:
import json

In [59]:
target_bigramms = dict(target_bigramms)

In [67]:
with open('w2v_all_clean_collocations.json', 'w', encoding = 'utf-8') as outfile:
    json.dump(target_bigramms, outfile, 
              ensure_ascii=False, 
              indent=4)